<center> <h2> Natural Language Processing</h2> </center>
<center> <h2> Text Classification using Scikit-Learn </h2> </center>

<br><br><br>

<h3>Overview:</h3>

<pre>
The main objective of this project is to completely understand the text classification problem using Machine Learning toolkit (scikit-learn). In this project, we are going to use the unstructured dataset which consists of comments extracted from twitter. The dataset consist of input attribute (Comment) and ouput attribute (Offensive). Dataset is labelled. The task is to predict offensive comment by using Machine Learning algorithms. 
</pre>

<h3>Machine Learning Approach:</h3>

<pre>
The problem of Text Classification is treated as a supervised learning approach because we going to use the labelled dataset.</pre>

<h3>The Input and Output are:</h3>

<pre>
<b> - Input: </b> Comment
<b> - Output:</b> Offensive (1/0)
<b> - Goal:  </b> Learn from Input to predict Output
</pre>

<h3> Three Phases of Machine Learning: </h3>

<pre>
<b>1. Training Phase </b> 
                    Learn from Training Dataset.
<b>2. Testing/Validation/Evaluation Phase</b>
                                         Evaluate how well the algorithm learned.
<b>3. Application Phase</b>
                       Use your learned/trained models in real world application.
</pre>

<br>

<center><h2> Table of Contents </h2></center>

<br>

<h3>PHASE 1 & 2: TRAINING AND TESTING </h3>

<pre>
<b>Step 1:</b> Import Libraries
<b>Step 2:</b> Read, Understand and Pre-process Train/Test Data
       <b>2.1</b> Read Data
       <b>2.2</b> Understand Data
       <b>2.3</b> Pre-process Data
       <b>2.4</b> Tokenization and Removal of Stop Words
<b>Step 3:</b> Label Encoding for Train/Test Data
<b>Step 4:</b> Feature Extraction – Changing Representation of features “from String to Feature-Vector”
<b>Step 5:</b> Train Machine Learning Algorithms using Training Data
<b>Step 6:</b> Evaluate Machine Learning Algorithms using Test Data
<b>Step 7:</b> Selection of Best Model
</pre>

<h3> PHASE 3: APPLICATION PHASE</h3>

<pre>
<b>Step 8:</b> Application in Real World
       <b>8.1</b> Combine Dataset (Train + Test)
       <b>8.2</b> Train Best Model on all data (Train + Test)
       <b>8.3</b> Save the Trained Model in Pickle File
<b>Step 9:</b> Make predictions on unseen/new data
       <b>9.1</b> Load the Trained Model
       <b>9.2</b> Take Input from User
       <b>9.3</b> Convert User Input into Feature Vector (Same as Feature Vector of Trained Model)
       <b>9.4</b> Apply Trained Model on Feature Vector of Unseen Dataset and give Output Prediction to the User
</pre>

<h5><b>Step 1:</b> Import Libraries</h5>

In [1]:
import re
import string
import nltk
import scipy
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

from IPython.display import display, HTML
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from prettytable import PrettyTable
from joblib import dump, load
from astropy.table import Table, Column

import urduhack
from urduhack.preprocess import replace_urls
from urduhack.preprocess import replace_numbers
from urduhack.preprocess import remove_punctuation
from urduhack.preprocess import remove_accents
from urduhack.preprocess import remove_english_alphabets
from urduhack.preprocess import normalize_whitespace
from urduhack.normalization import normalize
#from urduhack.tokenization import word_tokenizer

import demoji
demoji.download_codes()

from tkinter import *

... OK (Got response in 2.76 seconds)
Writing emoji data to C:\Users\Rabee Bin Amir\.demoji/codes.json ...
... OK


<h5><b>Step 2:</b> Read, Understand and Pre-process Train/Test Data</h5>

<pre><b>Step 2.1: Read Data </b></pre>

In [2]:
dataset = pd.read_csv("fyp_dataset.csv")

<pre><b>Step 2.2: Understand Data </b></pre>

In [3]:
print('\n')

dataset.columns.name = 'index'

print('Number of instances in Dataset:        %d'%(len(dataset)))
print('==============================',end='\n\n')

print('\n\n')



Number of instances in Dataset:        10138






<pre><b>Step 2.3: Pre-Process Data </b></pre>

In [4]:
print('\n')

def clean_comment(comment):
    
    comment = demoji.replace(comment,"")
    
    comment = re.sub(r'@[A-Za-z0-9]+', '', comment)
    
    comment = re.sub(r'#[A-Za-z0-9]*', '', comment)
    
    comment = replace_urls(comment,replace_with="")
    
    comment = replace_numbers(comment,replace_with="")
    
    comment = remove_accents(comment)
    
    comment = remove_english_alphabets(comment)
    
    comment = normalize_whitespace(comment)
    
    comment = remove_punctuation(comment)
    
    comment = normalize(comment)
    
    return comment

for i in dataset.index:
    a = dataset['comment'][i] 
    b = clean_comment(a)
    dataset.loc[[i],['comment']] = b
    

print('\n')

<pre><b>Step 2.4: Tokenization and Removal of Stop Words </b></pre>

In [5]:
stop_words = set("""

 آ آئی آئیں آئے آتا آتی آتے آداب آدھ آدھا آدھی آدھے آس آمدید آنا آنسہ آنی آنے
 آپ آگے آہ آہا آیا اب ابھی ابے اتوار ارب اربویں ارے اس اسکا اسکی اسکے اسی اسے اف
 افوہ الاول البتہ الثانی الحرام السلام الف المکرم ان اندر انکا انکی انکے انہوں انہی انہیں اوئے اور
 اوپر اوہو اپ اپنا اپنوں اپنی اپنے اپنےآپ اکبر اکثر اگر اگرچہ اگست اہاہا ایسا ایسی ایسے ایک بائیں
 بار بارے بالکل باوجود باہر بج بجے بخیر برسات بشرطیکہ بعد بعض بغیر بلکہ بن بنا بناؤ بند
 بڑی بھر بھریں بھی بہار بہت بہتر بیگم تاکہ تاہم تب تجھ تجھی تجھے ترا تری تلک تم تمام
 تمہارا تمہاروں تمہاری تمہارے تمہیں تو تک تھا تھی تھیں تھے تہائی تیرا تیری تیرے تین جا جاؤ
 جائیں جائے جاتا جاتی جاتے جانی جانے جب جبکہ جدھر جس جسے جن جناب جنہوں جنہیں جو جہاں جی
 جیسا جیسوں جیسی جیسے جیٹھ حالانکہ حالاں حصہ حضرت خاطر خالی خدا خزاں خواہ خوب خود دائیں درمیان
 دریں دو دوران دوسرا دوسروں دوسری دوشنبہ دوں دکھائیں دگنا دی دیئے دیا دیتا دیتی دیتے دیر دینا دینی
 دینے دیکھو دیں دیے دے ذریعے رکھا رکھتا رکھتی رکھتے رکھنا رکھنی رکھنے رکھو رکھی رکھے رہ رہا
 رہتا رہتی رہتے رہنا رہنی رہنے رہو رہی رہیں رہے ساتھ سامنے ساڑھے سب سبھی سراسر سلام سمیت سوا
 سوائے سکا سکتا سکتے سہ سہی سی سے شام شاید شکریہ صاحب صاحبہ صرف ضرور طرح طرف طور
 علاوہ عین فروری فقط فلاں فی قبل قطا لئے لائی لائے لاتا لاتی لاتے لانا لانی لانے لایا لو
 لوجی لوگوں لگ لگا لگتا لگتی لگی لگیں لگے لہذا لی لیا لیتا لیتی لیتے لیکن لیں لیے
 لے ماسوا مت مجھ مجھی مجھے محترم محترمہ محترمی محض مرا مرحبا مری مرے مزید مس مسز مسٹر مطابق
 مطلق مل منٹ منٹوں مکرمی مگر مگھر مہربانی میرا میروں میری میرے میں نا نزدیک نما نو نومبر
 نہ نہیں نیز نیچے نے و وار واسطے واقعی والا والوں والی والے واہ وجہ ورنہ وعلیکم وغیرہ ولے
 وگرنہ وہ وہاں وہی وہیں ویسا ویسے ویں پاس پایا پر پس پلیز پون پونا پونی پونے پھاگن
 پھر پہ پہر پہلا پہلی پہلے پیر پیچھے چاہئے چاہتے چاہیئے چاہے چلا چلو چلیں چلے چناچہ چند چونکہ
 چوگنی چکی چکیں چکے چہارشنبہ چیت ڈالنا ڈالنی ڈالنے ڈالے کئے کا کاتک کاش کب کبھی کدھر کر
 کرتا کرتی کرتے کرم کرنا کرنے کرو کریں کرے کس کسی کسے کل کم کن کنہیں کو کوئی کون
 کونسا کونسے کچھ کہ کہا کہاں کہہ کہی کہیں کہے کی کیا کیسا کیسے کیونکر کیونکہ کیوں کیے
 کے گئی گئے گا گرما گرمی گنا گو گویا گھنٹا گھنٹوں گھنٹے گی گیا ہائیں ہائے ہاڑ ہاں ہر
 ہرچند ہرگز ہزار ہفتہ ہم ہمارا ہماری ہمارے ہمی ہمیں ہو ہوئی ہوئیں ہوئے ہوا ہوبہو ہوتا ہوتی
 ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیلو ہیں ہے یا یات یعنی یک یہ یہاں یہی یہیں


""".split())

filtered_sentences = [] 

for i in dataset.index:
    token = dataset['comment'][i].split()
    for w in token: 
        if w not in stop_words: 
            filtered_sentences.append(w) 
     

<h5><b>Step 3:</b> Label Encoding for Train/Test Data</h5>

<h5><b>Step 4:</b> Feature Extraction</h5>

In [6]:
print('\n')

vector = TfidfVectorizer(strip_accents='unicode',
                         analyzer = 'word',
                         ngram_range = (1,3),
                         max_features = 5000)

all_data_features = vector.fit_transform(filtered_sentences)

all_data_features_names = vector.get_feature_names()


text1 = []
for i in dataset.index:
    text1.append(dataset["comment"][i])
X1 = vector.transform(text1)
df1 = pd.DataFrame(X1.toarray(), columns = all_data_features_names)
offensive1 = []
for y in dataset.index:
    offensive1.append(dataset["offensive"][y])
df1["offensive"] = offensive1    
df1.columns.name = 'index'

print('\n')

<h5><b>Step 5:</b> Split Input Vectors and Labels</h5>

In [7]:
X = X1
y = dataset.offensive
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=43)

<h5><b>Step 5:</b> Train Machine Learning Algorithms using Training Data</h5>

In [8]:
model_names = []

In [9]:
print('\n')

logistic_regression_cv = LogisticRegressionCV(max_iter = 1000)
warnings.filterwarnings("ignore", category=FutureWarning)
logistic_regression_cv.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(logistic_regression_cv)
model_names.append('LogisticRegressionCV')

print('\n')



Parameters and their values:

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=1000, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)




In [10]:
print('\n')

k_neighbors_classifier = KNeighborsClassifier()
warnings.filterwarnings("ignore", category=FutureWarning)
k_neighbors_classifier.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(k_neighbors_classifier)
model_names.append('KNeighborsClassifier')

print('\n')



Parameters and their values:

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')




In [11]:
print('\n')

bernoulli_nb = BernoulliNB()
warnings.filterwarnings("ignore", category=FutureWarning)
bernoulli_nb.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(bernoulli_nb)
model_names.append('BernoulliNB')

print('\n')



Parameters and their values:

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)




In [12]:
print('\n')

random_forest_classifier = RandomForestClassifier()
warnings.filterwarnings("ignore", category=FutureWarning)
random_forest_classifier.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(random_forest_classifier)
model_names.append('RandomForestClassifier')

print('\n')



Parameters and their values:

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)




In [13]:
print('\n')

linear_svc = LinearSVC(max_iter = 2000)
warnings.filterwarnings("ignore", category=FutureWarning)
linear_svc.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(linear_svc)
model_names.append('LinearSVC')

print('\n')



Parameters and their values:

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=2000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)




In [14]:
print('\n')

gradient_boosting_classifier = GradientBoostingClassifier()
warnings.filterwarnings("ignore", category=FutureWarning)
gradient_boosting_classifier.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(gradient_boosting_classifier)
model_names.append('GradientBoostingClassifier')

print('\n')



Parameters and their values:

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)




In [15]:
print('\n')

decision_tree_classifier = DecisionTreeClassifier()
warnings.filterwarnings("ignore", category=FutureWarning)
decision_tree_classifier.fit(X_train,np.ravel(y_train))

print("Parameters and their values:")
print("===========================",end='\n\n')
print(decision_tree_classifier)
model_names.append('CART')

print('\n')



Parameters and their values:

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')




<h5><b>Step 6:</b> Evaluate Machine Learning Algorithms using Test Data</h5>

In [16]:
Scores1 = []
Scores2 = []
Scores3 = []
Scores4 = []

In [17]:
print('\n')

y_prediction    = logistic_regression_cv.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)

Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

In [18]:
print('\n')

y_prediction    = k_neighbors_classifier.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)


Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

In [19]:
print('\n')

y_prediction    = bernoulli_nb.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)

Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

In [20]:
print('\n')

y_prediction    = random_forest_classifier.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)


Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

In [21]:
print('\n')

y_prediction    = linear_svc.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)


Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

In [22]:
print('\n')

y_prediction    = gradient_boosting_classifier.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)

Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

In [23]:
print('\n')

y_prediction    = decision_tree_classifier.predict(X_test)

Accuracy_Score  = round(accuracy_score(y_test,y_prediction),2)
Precision_Score = round(precision_score(y_test, y_prediction),2)
Recall_Score    = round(recall_score(y_test, y_prediction),2)
F1_Score        = round(f1_score(y_test, y_prediction),2)

Scores1.append(Accuracy_Score)
Scores2.append(Precision_Score)
Scores3.append(Recall_Score)
Scores4.append(F1_Score)

print('\n')

<h5><b>Step 7:</b> Selection of Best Model</h5>

In [24]:
print('\n')

p = PrettyTable()
p.field_names = ["Model", "Accuracy", "Precision", "Recall", "F1"]
p.add_row([model_names[0],Scores1[0],Scores2[0],Scores3[0],Scores4[0]])
p.add_row([model_names[1],Scores1[1],Scores2[1],Scores3[1],Scores4[1]])
p.add_row([model_names[2],Scores1[2],Scores2[2],Scores3[2],Scores4[2]])
p.add_row([model_names[3],Scores1[3],Scores2[3],Scores3[3],Scores4[3]])
p.add_row([model_names[4],Scores1[4],Scores2[4],Scores3[4],Scores4[4]])
p.add_row([model_names[5],Scores1[5],Scores2[5],Scores3[5],Scores4[5]])
p.add_row([model_names[6],Scores1[6],Scores2[6],Scores3[6],Scores4[6]])

print('Detailed Performance of all the models:')
print('=======================================')
print(p)

print('\n\n')

print('Best Model:')
print('=======================================')
print('+--------------------------+----------+')
print('|          Model           | Accuracy |')
print('+--------------------------+----------+')
print('|    %s    |   %.2f   |'%(model_names[0],Scores1[0]))
print('+--------------------------+----------+')

print('\n')



Detailed Performance of all the models:
+----------------------------+----------+-----------+--------+------+
|           Model            | Accuracy | Precision | Recall |  F1  |
+----------------------------+----------+-----------+--------+------+
|    LogisticRegressionCV    |   0.93   |    0.91   |  0.95  | 0.93 |
|    KNeighborsClassifier    |   0.79   |    0.86   |  0.71  | 0.77 |
|        BernoulliNB         |   0.93   |    0.91   |  0.96  | 0.93 |
|   RandomForestClassifier   |   0.94   |    0.91   |  0.97  | 0.94 |
|         LinearSVC          |   0.92   |    0.91   |  0.93  | 0.92 |
| GradientBoostingClassifier |   0.93   |    0.91   |  0.97  | 0.94 |
|            CART            |   0.91   |    0.9    |  0.92  | 0.91 |
+----------------------------+----------+-----------+--------+------+



Best Model:
+--------------------------+----------+
|          Model           | Accuracy |
+--------------------------+----------+
|    LogisticRegressionCV    |   0.93   |
+----------

<h5>Step 8: Application in Real World</h5>

<pre><b>8.1 Combine Dataset (Train + Test)</b></pre>

In [25]:
print('\n')

training_testing_dataset_after_fe = df1

print('\n')

<pre><b>8.2 Train Best Model on all data (Train + Test)</b></pre>

In [26]:
print('\n')

X = training_testing_dataset_after_fe.drop('offensive', axis=1)
y = training_testing_dataset_after_fe['offensive']

warnings.filterwarnings("ignore", category=FutureWarning)
logistic_regression_cv.fit(X,np.ravel(y))
print(logistic_regression_cv)

print('\n')



LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=1000, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)




<pre><b>8.3 Save the Trained Model in Pickle File</b></pre>

In [27]:
saved_model = pickle.dumps(logistic_regression_cv)
saved_vector = pickle.dumps(vector)

<br><br>

<h5><b>Step 9:</b> Make predictions on unseen/new data</h5>

<br>

<pre><b>9.1 Load the Trained Model</b></pre>

In [28]:
BestClassifier = pickle.loads(saved_model)
BestVector = pickle.loads(saved_vector)

<pre><b>9.2 Take Input from User</b></pre>
<pre><b>9.3 Convert User Input into Feature Vector (Same as Feature Vector of Trained Model)</b></pre>
<pre><b>9.4 Apply Trained Model on Feature Vector of Unseen Data and give Output Prediction to the User</b></pre>

In [29]:
window = Tk()
window.title("Twitter Sentiment Analysis")
window.geometry('1300x350')
window.iconbitmap('Twitter-icon.ico')

def prediction():
    
    input_comment = entry.get()
    input_list = []
    input_list.append(input_comment)
    a = BestVector.transform(input_list)
    b = pd.DataFrame(a.toarray(), columns = BestVector.get_feature_names())
    prediction = BestClassifier.predict(b)
    if prediction == 1:
        output = "offensive"
    else:
        output = "non-offensive"
    
    newWindow = Toplevel(window)
    newWindow.title("Prediction")
    newWindow.geometry('300x200')
    newWindow.iconbitmap('Twitter-icon.ico')
    finishButton = Button(newWindow, text="Exit", padx=10, pady=10, font="Verdana 16 bold", width=12 ,borderwidth=5, \
                  command=window.destroy)
    finishButton.place(relx=.5, rely=.7, anchor="center")
    if (output == "offensive"):
        labelOffensive = Label(newWindow, text = "Offensive", font="Verdana 20 bold", bg="gray", fg="red")
        labelOffensive.pack()
    if (output == "non-offensive"):
        labelNonOffensive = Label(newWindow, text = "Non-Offensive", font="Verdana 20 bold", bg="gray", fg="green")
        labelNonOffensive.pack()


    
labelFrame = LabelFrame(window, text="Enter the text for the prediction of gender:", padx=35, pady=30, font="Verdana 16 bold", \
                        borderwidth=10, height=20)
labelFrame.grid(row=0, column=0, columnspan=3)
labelFrame.place(relx=.5, rely=.3, anchor="center")

entry = Entry(labelFrame,width=78, borderwidth=5, justify="left", font="Verdana 16 bold")
entry.grid(row=1, column=0, columnspan=3, ipady=5)

btn1 = Button(window, text="Predict", padx=10, pady=10, font="Verdana 16 bold", width=12, \
              borderwidth=5, command=prediction)
btn1.place(relx=.2, rely=.7, anchor="center")

btn2 = Button(window, text="Exit", padx=10, pady=10, font="Verdana 16 bold", width=12 ,borderwidth=5, command=window.destroy)
btn2.place(relx=.8, rely=.7, anchor="center")



mainloop()